<a href="https://colab.research.google.com/github/hanbuck30/neural-layer/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim

In [98]:
timesteps = 5 # 시점의 수. NLP에서는 보통 문장의 길이
input_size = 3 # 입력의 차원. NLP에서는 보통 단어 벡터의 차원
hidden_size = 8 # 은닉 상태의 크기. 메모리 셀의 용량
output_size=5
batch_size=1
inputs = torch.randn(batch_size,timesteps,input_size)
hidden_state_t=torch.zeros(hidden_size,requires_grad=True)

In [99]:
Wx = torch.randn(hidden_size, input_size,requires_grad=True) 
Wh = torch.randn(hidden_size, hidden_size,requires_grad=True)
b = torch.randn(hidden_size,requires_grad=True)
learning_rate = 0.1

In [100]:
print(Wx.shape)
print(Wh.shape)
print(b.shape)

torch.Size([8, 3])
torch.Size([8, 8])
torch.Size([8])


In [101]:
inputs.shape

torch.Size([1, 5, 3])

In [102]:
Y=torch.LongTensor([[0,1,2,3,4]])
y_data=[Y]
Y.shape

torch.Size([1, 5])

In [103]:
b_fc=torch.randn(output_size,requires_grad=True)
w_fc=torch.randn(hidden_size, output_size,requires_grad=True)

In [104]:
total_hidden_state=[]
total_hidden_states=[]
def RNNmodel(inputs,hidden_state_t):
    def RNN(inputs,hidden_state_t):
        for i in inputs:
            for input_t in i: 
                output_t = torch.tanh(torch.matmul(Wx,input_t) + torch.matmul(Wh,hidden_state_t) + b)
                total_hidden_states.append(output_t) 
                hidden_state_t = output_t
            total_state = torch.stack(total_hidden_states,dim=0)
            total_hidden_state.append(total_state)
        total_state=torch.stack(total_hidden_state,dim=0)
        return total_state
    def fc(total_state):
        linear=torch.matmul(total_state,w_fc)+b_fc
        return linear
    v=fc(RNN(inputs,hidden_state_t))
    return v
RNNmodel(inputs,hidden_state_t).shape

torch.Size([1, 5, 5])

In [105]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam([Wx,Wh,b,b_fc,w_fc], learning_rate)

In [109]:
for i in range(100):
    total_hidden_state=[]
    total_hidden_states=[]
    optimizer.zero_grad()
    outputs=RNNmodel(inputs,hidden_state_t)
    loss = criterion(outputs.view(-1,output_size), Y.view(-1)) # view를 하는 이유는 Batch 차원 제거를 위해
    loss.backward() 
    optimizer.step() 

    
    result = outputs.data.numpy().argmax(axis=2) 
    print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data)

0 loss:  0.0005602967576123774 prediction:  [[0 1 2 3 4]] true Y:  [tensor([[0, 1, 2, 3, 4]])]
1 loss:  0.000558057043235749 prediction:  [[0 1 2 3 4]] true Y:  [tensor([[0, 1, 2, 3, 4]])]
2 loss:  0.0005559602868743241 prediction:  [[0 1 2 3 4]] true Y:  [tensor([[0, 1, 2, 3, 4]])]
3 loss:  0.0005537205142900348 prediction:  [[0 1 2 3 4]] true Y:  [tensor([[0, 1, 2, 3, 4]])]
4 loss:  0.0005516000092029572 prediction:  [[0 1 2 3 4]] true Y:  [tensor([[0, 1, 2, 3, 4]])]
5 loss:  0.0005494555225595832 prediction:  [[0 1 2 3 4]] true Y:  [tensor([[0, 1, 2, 3, 4]])]
6 loss:  0.0005473588244058192 prediction:  [[0 1 2 3 4]] true Y:  [tensor([[0, 1, 2, 3, 4]])]
7 loss:  0.0005452620098367333 prediction:  [[0 1 2 3 4]] true Y:  [tensor([[0, 1, 2, 3, 4]])]
8 loss:  0.000543189060408622 prediction:  [[0 1 2 3 4]] true Y:  [tensor([[0, 1, 2, 3, 4]])]
9 loss:  0.0005410684971138835 prediction:  [[0 1 2 3 4]] true Y:  [tensor([[0, 1, 2, 3, 4]])]
10 loss:  0.0005390431033447385 prediction:  [[0 1 2